In [1]:
import pysis
import numpy
from scipy.stats import _qmc
import pysis.flowsheet as sim_f
import win32
from pythoncom import com_error



In [2]:
class SimulacaoCalc:
    def __init__(self, fs: sim_f.Simulation):
        self.fs = fs
        self.Operations = fs.Operations
        self.materialStreams = fs.MatStreams
        self.distillationColumn = sim_f.DistillationColumn(
            fs.Operations["T-100"].COMObject
        )
        self.Pfr = sim_f.PFR(self.fs.Operations["PFR-100"].COMObject)
        # fs.Solver.CanSolve=True
        self.fs.set_visible(1)
        pass

    def set_visible(self):
        self.fs.set_visible(1)

    def DesignVariables(self):
        return {
            "VolumeReator": self.Pfr.get_volume(),
            "AreaTC": [
                self.fs.Operations[u].COMObject.UAValue * 1000 / 1700
                for u in self.Operations.keys()
                if self.fs.Operations[u].classification == "Heat Transfer Equipment"
                and any(
                    [
                        x == "HeatTransferArea"
                        for x in self.fs.Operations[u].COMObject.__dir__()
                    ]
                )
            ],
            "VazaoWasteWater": self.fs.MatStreams[
                "LIQ"
            ].COMObject.IdealLiquidVolumeFlowValue
            * 1000,
            "PotenciaFornalha": self.fs.EnerStreams["Q-105"].get_power() / 1e3,
            "VolumePackingTD": self.fs.Operations["CustoDestilação"]
            .COMObject.Cell("A1")
            .CellValue,
            "PotenciaCompressores": [
                self.fs.Operations[u].COMObject.EnergyValue
                for u in self.Operations.keys()
                if self.Operations[u].classification == "Rotating Equipment"
            ],
            "MassaProcessVessel": [
                self.fs.Operations["ProcessVessel"].COMObject.Cell("B7").CellValue,
                self.fs.Operations["ProcessVessel"].COMObject.Cell("B8").CellValue,
            ],
        }

    def listDesignVariables(self):
        return numpy.array(list(self.DesignVariables().values())).flatten()

    def UCEquipamentos(self):
        design_variables = self.DesignVariables()

        return {
            "CustoReator": 4.00
            * (61500 + 32500 * (design_variables["VolumeReator"]) ** 0.80),
            "CustoTrocadoresDeCalor": sum(
                [1.0 * (28000 + 54 * u**1.20) for u in design_variables["AreaTC"]]
            ),
            "CustoTorreResfriamento": 2.5
            * (170000 + 1500 * design_variables["VazaoWasteWater"] ** 0.90),
            "CustoFornalha": 2.5
            * (80000 + 10900 * (design_variables["PotenciaFornalha"]) ** 0.80),
            "CustoDestilação": 4000 * design_variables["VolumePackingTD"],
            "CustoComp": sum(
                [
                    2.5 * (580000 + 20000 * (u) ** 0.6)
                    for u in design_variables["PotenciaCompressores"]
                ]
            ),
            "CustoProcessVessel": sum(
                [
                    4.0 * (17400 + 79 * (u) ** 0.85)
                    for u in design_variables["MassaProcessVessel"]
                ]
            ),
        }

    def CustoEquipamentos(self):
        return sum(list(self.UCEquipamentos().values()))

    def QntdUtilidades(self):
        return {
            "VazaoUtilidadeFria": self.fs.MatStreams[
                "UtilidadeFria"
            ].COMObject.IdealLiquidVolumeFlowValue
            * 3600,
            "PotenciaAquecimento": self.fs.Operations["UtilidadeQuenteCusto"]
            .COMObject.Cell("B4")
            .CellValue
            / 1e6
            * 3600,
            "VazaoWasteWater": self.fs.MatStreams[
                "LIQ"
            ].COMObject.IdealLiquidVolumeFLowValue
            * 3600,
            "PotenciaEletricidade": self.fs.EnerStreams["CustoEletricidade"].get_power()
            / 1e3
        }

    def CustoUtilidades(self):
        qntdutilidades = self.QntdUtilidades()
        return {
            "CustoResfriamento": qntdutilidades["VazaoUtilidadeFria"] * 0.03,
            "CustoAquecimento": qntdutilidades["PotenciaAquecimento"] * 12.1,
            "CustoWasteWater": qntdutilidades["VazaoWasteWater"] * 1.5,
            "CustoEletricidade": qntdutilidades["PotenciaEletricidade"] * 94.5,
        }

    def CustoTotalUtilidades(self):
        return sum(list(self.CustoUtilidades().values()))

    def VazaoMP(self):
        return {
            "QntdH2": self.fs.MatStreams["9"].COMObject.IdealLiquidVolumeFlowValue
            * 3600,
            # ton
            "QntdCO2": self.fs.MatStreams["1"].get_massflow() / 1000,
        }

    def CustoMP(self):
        return {
            "CustoH2": self.VazaoMP()["QntdH2"] * 0.165,
            "CustoCO2": self.VazaoMP()["QntdCO2"] * 95.50,
        }

    def CustoTotalMP(self):
        return sum(list(self.CustoMP().values()))

    def simulated_values(self):
        return list(numpy.array(list(self.QntdUtilidades().values())).flatten()) + list(
            numpy.array(list(self.VazaoMP().values())).flatten()
        )

    def obj_f(self):
        return [
            self.CustoEquipamentos(),
            self.CustoTotalMP(),
            self.CustoTotalUtilidades(),
            self.fs.MatStreams["DestL"].get_compmolarflow()["Methanol"],
        ]

    def set_dof(self, x):
        self.fs.Solver.CanSolve = False

        self.materialStreams["13"].set_temperature(x[0], "C")

        self.materialStreams["9"].set_molarflow(x[1], "kgmole/h")

        self.fs.Operations["TEE-100"].SplitsValue = (x[2], 1 - x[2])

        self.distillationColumn.set_specifications(spec="Reflux Ratio", value=x[3])

        self.Pfr.set_volume(x[4])

        self.materialStreams["10"].set_pressure(x[5], "kpa")
        self.materialStreams["8"].set_pressure(x[5], "kpa")
        self.materialStreams["20"].set_pressure(x[5], "kpa")

        self.materialStreams["REC-2"].set_pressure(x[5], "kpa")
        self.fs.Solver.CanSolve = True

    def RReciclo(self):
        self.materialStreams["REC"].set_molarflow(5.204e4)
        self.materialStreams["REC"].set_compmolarflow(
            self.materialStreams["REC-2"].get_compmolarflow()
        )
        streams_pressure = dict(
            zip(
                list(self.fs.MatStreams.keys()),
                [
                    self.fs.MatStreams[u].get_pressure()
                    for u in list(self.fs.MatStreams.keys())
                ],
            )
        )

    def reset_stream(self):
        unconverged_streams = [
            i
            for i in list(self.fs.MatStreams.keys())
            if self.fs.MatStreams[i].COMObject.MassFlow() < 0
            or self.fs.MatStreams[i].COMObject.MolarEntropy() < 0
        ]
        streams_pressure = dict(
            zip(
                list(self.fs.MatStreams.keys()),
                [
                    self.fs.MatStreams[u].get_pressure()
                    for u in list(self.fs.MatStreams.keys())
                ],
            )
        )
        unconverged_streams_bool = any(
            [
                self.fs.MatStreams[i].COMObject.MassFlow() < 0
                or self.fs.MatStreams[i].COMObject.MolarEntropy() < 0
                for i in list(self.fs.MatStreams.keys())
            ]
        )
        for i in unconverged_streams:
            if self.fs.MatStreams[i].COMObject.CanModifyStream == True:
                self.fs.MatStreams[i].set_pressure(100)
                self.fs.MatStreams[i].set_pressure(streams_pressure[i])
    def reset_simulation(self):
        self.fs.close()
        self.fs=0
        self.fs=sim_f.Simulation(path=(r"C:\Users\LuanNogueira\Downloads\Ic-Metamodelagem\DWSIMSIMS\HidrogenaçãoCO2A.hsc"))
        self.fs.set_visible(1)
        pass

In [3]:
def reset_simulation(SimulatioCalculation):
    SimulatioCalculation.fs.close()
    SimulatioCalculation=0
    SimulatioCalculation=SimulacaoCalc(fs=pysis.Simulation(r"C:\Users\LuanNogueira\Downloads\Ic-Metamodelagem\DWSIMSIMS\HidrogenaçãoCO2A.hsc"))
    pass


In [4]:
def LHS(n, step):
    sampler = _qmc.LatinHypercube(d=6)
    sample = sampler.random(n)
    _qmc.discrepancy(sample)
    l_bounds = [180, 4500, 0.001, 1.25, 35, 4500]
    u_bounds = [240, 6500, 0.05, 1.8, 55, 5500]
    x = list(_qmc.scale(sample, l_bounds, u_bounds))
    # Retorna Lista com sublistas de tamanho step
    sublists = [x[i : i + step] for i in range(0, len(x), step)]
    return sublists

In [5]:
def run(sublists: list):
    y = []
    SimulacaoCalc1 = SimulacaoCalc(
        fs=pysis.Simulation(
            r"C:\Users\LuanNogueira\Downloads\Ic-Metamodelagem\DWSIMSIMS\HidrogenaçãoCO2A.hsc"
        )
    )
    f2=0
    f = 0
    for sublist in sublists:
        # SimulacaoCalc1.fs.update_flowsheet()

        for lists in sublist:
            f3=0
            try:
                SimulacaoCalc1.set_dof(x=lists)
                

                trials = 0
                is_converged = False
                is_valid = False
                while (
                    is_converged == False
                    or is_valid == False
                    or stream_convergeance == False
                ) and trials <= 3:
                    SimulacaoCalc1.set_dof(x=lists)
                    
                    print(f)
                    print("degrees of freedom: {}".format(*[lists]))
                    if trials > 0:
                        try:
                            SimulacaoCalc1.RReciclo()

                        except:
                            print("Resetando simulação")
                            SimulacaoCalc1.fs.close()
                            SimulacaoCalc1 = 0
                            SimulacaoCalc1 = SimulacaoCalc(
                                fs=pysis.Simulation(
                                    r"C:\Users\LuanNogueira\Downloads\Ic-Metamodelagem\DWSIMSIMS\HidrogenaçãoCO2A.hsc"
                                )
                            )
                            SimulacaoCalc1.set_dof(lists)
                            SimulacaoCalc1.RReciclo()
                        else:
                            print("Resetando Reciclo e correntes")
                        try:
                            SimulacaoCalc1.reset_stream()
                        except AssertionError:
                            None
                    try:
                        sv = SimulacaoCalc1.simulated_values()
                        is_valid = all([i > -3000 for i in sv])
                    except:
                        # print("erro")
                        is_valid = False
                    flag_streams = dict(
                        list(
                            zip(
                                list(SimulacaoCalc1.fs.MatStreams.keys()),
                                [
                                    i >= 0
                                    for i in [
                                        SimulacaoCalc1.fs.MatStreams[f].get_massflow()
                                        for f in SimulacaoCalc1.fs.MatStreams.keys()
                                    ]
                                ],
                            )
                        )
                        + list(
                            zip(
                                list(SimulacaoCalc1.fs.EnerStreams.keys()),
                                [
                                    i >= 0
                                    for i in [
                                        f.get_power()
                                        for f in SimulacaoCalc1.fs.EnerStreams.values()
                                    ]
                                ],
                            )
                        )
                    )
                    stream_convergeance = all(
                        [
                            i >= 0
                            for i in [
                                f.get_massflow()
                                for f in SimulacaoCalc1.fs.MatStreams.values()
                            ]
                            + [
                                f.get_power()
                                for f in SimulacaoCalc1.fs.EnerStreams.values()
                            ]
                        ]
                    )
                    is_converged = SimulacaoCalc1.distillationColumn.get_convergence()
                    trials += 1
                # print(DesignVariables())
                # SimulacaoCalc1.reset_stream()
                
                y.append(
                    SimulacaoCalc1.obj_f()
                    + [is_valid, is_converged, stream_convergeance]
                )
                with open(r"DadosMetamodelagemY4.csv", mode="a") as file1:
                    u = (
                        str(j)
                        for j in SimulacaoCalc1.obj_f()
                        + [is_valid, is_converged, stream_convergeance]
                    )
                    for j in u:
                        file1.write(j + ",")
                    file1.write("\n")

                with open(r"DadosMetamodelagemX4", mode="a") as file2:
                    p = (str(i) for i in lists)
                    for t in p:
                        file2.write(t + ",")

                    file2.write("\n")
            
                f3+=1
                f += 1
                # raise com_error
            except:
                print('com_error, rodando list_error')
                list_error=sublists[(f2+1):]
                run(list_error)
                

        f2+=1
        SimulacaoCalc1.fs.close()
        SimulacaoCalc1 = 0
        SimulacaoCalc1 = SimulacaoCalc(
            fs=pysis.Simulation(
                r"C:\Users\LuanNogueira\Downloads\Ic-Metamodelagem\DWSIMSIMS\HidrogenaçãoCO2A.hsc"
            )
        )

In [6]:
run(LHS(5000,200))


0
degrees of freedom: [2.04033211e+02 4.91522685e+03 2.30917417e-03 1.35808196e+00
 5.02347614e+01 5.13728567e+03]
1
degrees of freedom: [2.02985751e+02 5.43952553e+03 3.92797333e-02 1.43365813e+00
 3.57208672e+01 5.28686000e+03]
2
degrees of freedom: [2.04195930e+02 5.50155269e+03 3.42429512e-03 1.69357887e+00
 3.82522887e+01 4.85106422e+03]
3
degrees of freedom: [2.32449561e+02 5.84586503e+03 7.92690752e-03 1.33470089e+00
 5.08626882e+01 5.11862372e+03]
4
degrees of freedom: [2.01863163e+02 6.40735309e+03 1.80457460e-02 1.58456029e+00
 4.86608641e+01 4.89312895e+03]
5
degrees of freedom: [1.89537089e+02 5.97240451e+03 1.57658525e-02 1.34785018e+00
 4.01960757e+01 5.04970304e+03]
6
degrees of freedom: [2.21962296e+02 5.60119846e+03 2.21852673e-02 1.44005629e+00
 4.38469922e+01 4.89865701e+03]
7
degrees of freedom: [2.24527174e+02 5.18276791e+03 1.51251245e-02 1.76625011e+00
 3.84035311e+01 4.97036340e+03]
8
degrees of freedom: [2.23955131e+02 5.70097558e+03 1.14293026e-02 1.54957546e+

com_error: (-2147467262, 'Não há suporte para esta interface', None, None)

In [ ]:
sc=SimulacaoCalc(fs=sim_f.Simulation(r"C:\Users\LuanNogueira\Downloads\Ic-Metamodelagem\DWSIMSIMS\HidrogenaçãoCO2A.hsc"))

In [ ]:
sc.CustoTotalUtilidades()

62838.100905047126